# Dividend Aristocrats

In this notebook, I am going to generate a list of companies called Dividend Aristocrats. There are some requirements for companies to be selected in the group of Dividend Aristocrats, I will make the process simple by selecting companies in S&P 500 which have paid out dividends at least 25 years. I can add more requirements on top of the threshold of 25 years.

## Get Stock Symbols from S&P 500

In [24]:
import yfinance as yf
import pandas as pd
import datetime as dt

In [13]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [14]:
sp500 = data[0]
sp500 = sp500[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date first added']]
sp500.head(10)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Date first added
0,MMM,3M Company,Industrials,Industrial Conglomerates,1976-08-09
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,1964-03-31
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,2012-12-31
3,ABMD,Abiomed,Health Care,Health Care Equipment,2018-05-31
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,2011-07-06
5,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,2015-08-31
6,ADBE,Adobe Inc.,Information Technology,Application Software,1997-05-05
7,AMD,Advanced Micro Devices,Information Technology,Semiconductors,2017-03-20
8,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,2015-07-09
9,AES,AES Corp,Utilities,Independent Power Producers & Energy Traders,1998-10-02


In [60]:
sp500_symbols = sp500['Symbol'].values.tolist()
sp500_sectors = list(set(sp500['GICS Sector'].values.tolist()))

sp500_symbols_new = []

for symbol in sp500_symbols:
    if '.' in symbol:
        sp500_symbols_new.append(symbol.replace('.','-'))
    else:
        sp500_symbols_new.append(symbol)

sp500_symbols = sp500_symbols_new        

## Retrieve Dividend Data

In [70]:
companies = {}
#sp500_symbols = ['AAPL', 'JNJ']
for symbol in sp500_symbols:
    
    prices = yf.Ticker(symbol).history(period='max')
    dividends = prices[prices['Dividends'] > 0]
    if len(dividends) > 0:
        first_year = dividends.index[0].year
    else:
        continue
    last_year = dt.datetime.today().year
    annual_div = {}

    for year in range(first_year, last_year):
        div_sum = dividends[dividends.index.year ==  year]['Dividends'].sum()
        if div_sum == 0: # dividend discontinued --> no consistency
            break
        else:
            annual_div[year] = div_sum

    if len(annual_div) >= 25: # dividend aristocrats
        companies[symbol] = []
        companies[symbol].append(annual_div)
        companies[symbol].append(len(annual_div))

In [71]:
companies

{'MMM': [{1970: 0.082,
   1971: 0.115752,
   1972: 0.120376,
   1973: 0.13125,
   1974: 0.1565,
   1975: 0.16875,
   1976: 0.181,
   1977: 0.2125,
   1978: 0.25,
   1979: 0.3,
   1980: 0.35,
   1981: 0.375,
   1982: 0.4,
   1983: 0.4125,
   1984: 0.425,
   1985: 0.4375,
   1986: 0.45,
   1987: 0.465,
   1988: 0.53,
   1989: 0.65,
   1990: 0.73,
   1991: 0.78,
   1992: 0.8,
   1993: 0.83,
   1994: 0.88,
   1995: 0.94,
   1996: 2.16,
   1997: 1.06,
   1998: 1.1,
   1999: 1.12,
   2000: 1.16,
   2001: 1.2,
   2002: 1.24,
   2003: 1.32,
   2004: 1.44,
   2005: 1.68,
   2006: 1.84,
   2007: 1.92,
   2008: 2.0,
   2009: 2.04,
   2010: 2.1,
   2011: 2.2,
   2012: 2.36,
   2013: 2.54,
   2014: 3.42,
   2015: 4.1,
   2016: 4.44,
   2017: 4.7,
   2018: 5.44,
   2019: 5.76,
   2020: 5.88},
  51],
 'ABT': [{1983: 0.026934,
   1984: 0.032265,
   1985: 0.037877,
   1986: 0.045172000000000004,
   1987: 0.03984,
   1988: 0.06453,
   1989: 0.075751,
   1990: 0.09034,
   1991: 0.107735,
   1992: 0.12906

## List of Dividend Kings

In [91]:
div_kings = {}
aristocrats = {}
for company in companies.keys():
    years = companies[company][1]
    if years >= 50:
        div_kings[company] = years
    else:
        aristocrats[company] = years

In [92]:
div_kings

{'MMM': 51,
 'MO': 59,
 'AEP': 51,
 'BA': 59,
 'CNP': 59,
 'KO': 59,
 'ED': 59,
 'DTE': 59,
 'XOM': 59,
 'GE': 59,
 'HON': 51,
 'HWM': 59,
 'HPQ': 56,
 'IBM': 59,
 'IP': 59,
 'JNJ': 59,
 'MRO': 59,
 'MRK': 51,
 'PG': 59,
 'RTX': 51}

In [90]:
div_kings_df = pd.DataFrame.from_dict(div_kings, orient='index', columns=['Consecutive Years of Dividend Payment'])
div_kings_df.sort_values(by = ['Consecutive Years of Dividend Payment'], ascending=False).head(20)

,Consecutive Years of Dividend Payment
XOM,59
GE,59
JNJ,59
IP,59
IBM,59
HWM,59
MO,59
MRO,59
DTE,59
ED,59


In [96]:
len(div_kings)

20

## List of Dividend Aristocrats

In [94]:
aristocrats_df = pd.DataFrame.from_dict(aristocrats, orient='index', columns=['Consecutive Years of Dividend Payment'])
aristocrats_df.sort_values(by = ['Consecutive Years of Dividend Payment'], ascending=False).head(20)

,Consecutive Years of Dividend Payment
PEP,49
TXN,49
EMR,49
DE,49
HAL,49
LLY,49
DD,49
BMY,49
PFE,49
ETN,49


In [95]:
len(aristocrats)

183